# DSCI 525: Milestone 1
In this notebook, we download rainfall data from [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681) in a zip file, extract all the csv files and combine them into a single file. The comparison of time taken to combine these files across different laptops is provided. EDA is done on the combined file and techniques used to reduce memory usage are implemented in python. In the end, the dataframe is transferred from python to R using `enter-approach-name`.

## 1. Importing packages

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

## 2. Download data with the figshare API 

In [16]:
current_path = os.getcwd()
current_path

'/Users/rohitrawat/MDS/Block6/525/week-1'

Here, we specify the article we want to import from `figshare` with the website url and the output directory.

In [4]:
article_id = 14096681  
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [5]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

We download the file `data.zip` to our local computer.

In [8]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 10.8 s, sys: 11.5 s, total: 22.2 s
Wall time: 31min 57s


Now, we unzip all the csv files from the zipped file.

In [9]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 14.6 s, sys: 1.32 s, total: 15.9 s
Wall time: 16.8 s


In [10]:
%%time
### just listing to get an idea how individual file looks like 
df_1 = pd.read_csv("figsharerainfall/ACCESS-CM2_daily_rainfall_NSW.csv")
df_1

CPU times: user 868 ms, sys: 113 ms, total: 981 ms
Wall time: 1.03 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
0,1889-01-01 12:00:00,-36.25,-35.00,140.625,142.50,3.293256e-13
1,1889-01-02 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
2,1889-01-03 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
3,1889-01-04 12:00:00,-36.25,-35.00,140.625,142.50,0.000000e+00
4,1889-01-05 12:00:00,-36.25,-35.00,140.625,142.50,1.047658e-02
...,...,...,...,...,...,...
1932835,2014-12-27 12:00:00,-30.00,-28.75,151.875,153.75,2.951144e-02
1932836,2014-12-28 12:00:00,-30.00,-28.75,151.875,153.75,2.257118e-01
1932837,2014-12-29 12:00:00,-30.00,-28.75,151.875,153.75,1.204670e-01
1932838,2014-12-30 12:00:00,-30.00,-28.75,151.875,153.75,2.632404e-02


Removing the file named `observed_daily_rainfall_SYD.csv` in the data folder.

In [13]:
os.remove(output_directory + 'observed_daily_rainfall_SYD.csv')

## 3. Combining CSV files

Changing the directory to enter the folder with the csv files.

In [17]:
os.chdir(current_path + '/figsharerainfall/')

In [18]:
%%time
import pandas as pd
use_cols = df_1.columns
files = glob.glob('*.csv')
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
                for file in files)
              )
df.to_csv("combined_data.csv")

CPU times: user 5min 22s, sys: 9.8 s, total: 5min 32s
Wall time: 5min 36s


In [21]:
%%sh
du -sh combined_data.csv

5.6G	combined_data.csv


### <center >Comparison of time taken to load and combine the files
    
| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Mel         |                  |     |           |        |            |
| Rohit       |  macOS Monterey  | 8 GB| M1, 8-core |   Yes |  5min 22s  |
| Rowan       |                  |     |           |        |            | 

### EDA on the combined data set

In [22]:
df.shape

(62467843, 6)

In [23]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
